In [3]:


import os
import pandas as pd
import numpy as np
import glob
import itertools 
from ray import tune
import subprocess


Repo='/home/berkeleylab/Repository/beam/'
Current='/home/berkeleylab/Calib_documentation/TUNE_integrate/TUNE_test'

In [6]:
#new_param = "= 9.9 \n"

config={"new_param": tune.grid_search([])}

def ext_change(param):    
    if param == 'edit':
        for filename in glob.iglob(os.path.join(Repo, 'test/input/beamville/beam.conf')):
            os.rename(filename, filename[:-9] + 'beam.txt')
    elif param == 'save':
        for filename in glob.iglob(os.path.join(Repo, 'test/input/beamville/beam.txt')):
            os.rename(filename, filename[:-8] + 'beam.conf')

def change_conf(new_param=config["new_param"]):
    with open(Repo+'test/input/beamville/beam.txt', 'r') as fin:
        file_text=fin.readlines()
    for i in range(24,32,1):
        file_text[i] = file_text[i].split('=',1)[0]+'= '+str(new_param)+' \n'
    with open(Repo+'test/input/beamville/beam.txt', 'w') as fini:
        for i in file_text:
            fini.write(i)
        
      
def run_beam():
    os.chdir(Repo)
    subprocess.call([Repo+'runme.sh'])
    os.chdir(Current)

def all_subdirs_of(b='/home/berkeleylab/Repository/beam/output/beamville/'):
  result = []
  for d in os.listdir(b):
    bd = os.path.join(b, d)
    if os.path.isdir(bd): result.append(bd)
  return result





In [3]:

def beamville(config):
    ext_change('edit')
    change_conf(new_param=config["new_param"])
    ext_change('save') 
    run_beam()
    latest_subdir = max(all_subdirs_of(), key=os.path.getmtime)
    df = pd.read_csv(latest_subdir+"/referenceRealizedModeChoice.csv").iloc[[0, -1]].drop(['iterations'], axis=1)
    acc = (df.iloc[0] - df.iloc[-1]).abs().sum()/df.iloc[0].sum()
    tune.track.log(mean_accuracy=acc)
    

analysis = tune.run(beamville, config={"new_param": tune.grid_search([2.0, 3.0])})


2019-12-10 16:18:48,384	INFO resource_spec.py:205 -- Starting Ray with 13.33 GiB memory available for workers and up to 6.68 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2019-12-10 16:18:50,435	INFO function_runner.py:254 -- tune.track signature detected.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/13.33 GiB heap, 0.0/4.59 GiB objects
Memory usage on this node: 9.8/31.6 GiB



2019-12-10 16:18:54,605	ERROR log_sync.py:28 -- Log sync requires rsync to be installed.
2019-12-10 16:18:54,665	ERROR log_sync.py:28 -- Log sync requires rsync to be installed.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/8 CPUs, 0/0 GPUs, 0.0/13.33 GiB heap, 0.0/4.59 GiB objects
Memory usage on this node: 10.4/31.6 GiB
Result logdir: /home/berkeleylab/ray_results/beamville
Number of trials: 2 ({'RUNNING': 1, 'PENDING': 1})
PENDING trials:
 - beamville_1_new_param=3.0:	PENDING
RUNNING trials:
 - beamville_0_new_param=2.0:	RUNNING

(pid=9802) Starting a Gradle Daemon, 1 busy and 3 stopped Daemons could not be reused, use --status for details
(pid=9796) [-Xmx140g, -Xms70g, -XX:+UseParallelGC, -XX:+UseParallelOldGC, -XX:MetaspaceSize=150M, -Djava.awt.headless=true, -Dlogback.configurationFile=logback_prod.xml, -XX:+PrintGCDetails, -XX:+PrintGCDateStamps, -Xloggc:gc_12-11-2019_00-18-58.log]
(pid=9796) :compileJava NO-SOURCE
(pid=9796) :compileScala
(pid=9796)  UP-TO-DATE
(pid=9796) :processResources UP-TO-DATE
(pid=9796) :classes UP-TO-DATE
(pid=9796) :run
(pid=9796) 
(pid=9796) 
(pid=9796)   ________
(pid=9796)   ___  __ )__________ _____

2019-12-10 16:20:07,510	INFO tune.py:276 -- Returning an analysis object by default. You can call `analysis.trials` to retrieve a list of trials. This message will be removed in future versions of Tune.


Result for beamville_0_new_param=2.0:
  date: 2019-12-10_16-20-07
  done: false
  experiment_id: 266f02b98e524368a34f9686cbf00f3a
  hostname: KiranCHHATRE
  iterations_since_restore: 1
  mean_accuracy: 0.9499702911467618
  node_ip: 128.3.105.143
  pid: 9802
  time_since_restore: 72.61442804336548
  time_this_iter_s: 72.61442804336548
  time_total_s: 72.61442804336548
  timestamp: 1576023607
  timesteps_since_restore: 0
  training_iteration: 0
  trial_id: cf11c466
  
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/8 CPUs, 0/0 GPUs, 0.0/13.33 GiB heap, 0.0/4.59 GiB objects
Memory usage on this node: 13.7/31.6 GiB
Result logdir: /home/berkeleylab/ray_results/beamville
Number of trials: 2 ({'RUNNING': 1, 'TERMINATED': 1})
RUNNING trials:
 - beamville_0_new_param=2.0:	RUNNING, [1 CPUs, 0 GPUs], [pid=9802], 72 s, 0 iter, 0.95 acc
TERMINATED trials:
 - beamville_1_new_param=3.0:	TERMINATED, [1 CPUs, 0 GPUs], [pid=9796], 66 s, 0 iter, 0.968 acc

== Status ==
Using FIFO sch

In [4]:

print("Best config: ", analysis.get_best_config(metric="mean_accuracy"))


Best config:  {'new_param': 3.0}


In [5]:
analysis.dataframe()


,mean_accuracy,trial_id,training_iteration,time_this_iter_s,done,timesteps_total,episodes_total,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/new_param,logdir
0,0.949970,cf11c466,0,72.614428,False,NaN,NaN,266f02b98e524368a34f9686cbf00f3a,2019-12-10_16-20-07,1576023607,72.614428,9802,KiranCHHATRE,128.3.105.143,72.614428,0,1,2.0,/home/berkeleylab/ray_results/beamville/beamvi...
1,0.968152,cf16864a,0,66.360611,False,NaN,NaN,32a2ccfc8f274654b755380900152b0c,2019-12-10_16-20-01,1576023601,66.360611,9796,KiranCHHATRE,128.3.105.143,66.360611,0,1,3.0,/home/berkeleylab/ray_results/beamville/beamvi...
